### Introdoction

Source: https://github.com/guidance-ai/guidance/blob/main/notebooks/tutorials/intro_to_guidance.ipynb

In [2]:
from guidance import models

MODEL_PATH = r"mistral-7b-instruct-v0.2.Q4_K_M.gguf"

mistral = models.LlamaCpp(
    MODEL_PATH,
    n_ctx=4096,
    n_threads=10
    )

In [13]:
llm = mistral + "what is blinrog offset mean?"

In [14]:
from guidance import gen

llm + gen()

In [8]:
mistral + '''\
Q: What is the most populous country in the world?
A:''' + gen(stop="Q:")

In [9]:
# using template with f string
query = "When is the World War 2 start?"
mistral + f'''\
Q: {query}
A:{gen(stop="Q:")}'''

In [11]:
# Capturing specific portions of the output into variable
query = "When is the World War 2 start?"
llm = mistral + f'''\
Q: {query}
A:{gen(name="answer", stop="Q:")}'''

In [12]:
llm['answer']

' World War 2 started on September 1, 1939. It began when Germany, under Adolf Hitler, invaded Poland. This event marked the beginning of a major global conflict that lasted until September 2, 1945. The war involved many countries, and it resulted in significant loss of life and destruction. It is important to note that there were earlier conflicts and tensions that contributed to the outbreak of World War 2, such as the Treaty of Versailles and the rise of totalitarian regimes in Europe. However, September 1, 1939, is generally considered the official start date of the war.'

In [14]:
# function encapsulation
import guidance

@guidance
def qa_bot(llm, query):
    llm += f'''\
    Q: {query}
    A: {gen(name="answer", stop="Q:")}'''
    return llm

query = "When is the World War 1 start? Why it named World War 1?"
mistral + qa_bot(query) # note we don't pass the `lm` arg here (that will get passed during execution when it gets added to the model)

In [15]:
# select among choices
from guidance import select

query = "the room is small and stinky"
mistral + f'''\
Q: {query}
The query sentiment is either positive or negative.
Choice: {select(['POSITIVE', 'NEGATIVE'], name="choice")}''' 

In [18]:
# constrained generation with conditional
@guidance
def qa_bot(llm, query):
    llm += f'''\
    Q: {query}
    Now I will choose to either SEARCH the web or RESPOND.
    Choice: {select(["SEARCH", 'RESPOND'], name="choice")}
    '''
    if llm["choice"] == "SEARCH":
        llm += "A: I don't know, you can google it yourself"
    else:
        llm += f'A: {gen(stop="Q", name="answer")}'
    return llm

In [19]:
query = "What is the capital city of Russia?"
mistral + qa_bot(query)

In [24]:
query = "Who won the last Kentucky derby and by how much?"
mistral + qa_bot(query)

In [32]:
# Generating list
query = "Indonesia"

llm = mistral + f'''\
Give random valid provinces name of {query}.
'''

llm += f"Here are 3 provinces of {query}\n"
for i in range(3):
    llm += f'''{i+1}. "{gen(stop='"', name="queries", list_append=True)}"\n''' # the 'queries' is variable name for the list

In [ ]:
# Generating list
query = "Indonesia"

llm = mistral + f'''\
Give random valid provinces name of {query}.
'''

llm += f"Here are 3 provinces of {query}\n"
for i in range(3):
    llm += f'''{4-i}. "{gen(stop='"', name="queries", list_append=True)}"\n''' # the 'queries' is variable name for the list

In [50]:
# print the list
llm['queries']

['Jawa Barat', 'Sulawesi Utara', 'Bali']

In [55]:
# chat
from guidance import system, user, assistant

mistral_chat = models.LlamaCppChat(
    MODEL_PATH,
    n_ctx=4096,
    n_threads=10
    ) 

with system():
    llm = mistral_chat + "you are helpful assistant"

with user():
    llm += "What is the meaning of life?"

with assistant():
    llm += gen("response")

In [56]:
# multiturn chat
from guidance import system, user, assistant

mistral_chat = models.LlamaCppChat(
    MODEL_PATH,
    n_ctx=4096,
    n_threads=10
    ) 

@guidance
def experts(lm, query):
    with system():
        lm += "You are a helpful assistant."

    with user():
        lm += f"""\
        I want a response to the following question:
        {query}
        Who are 3 world-class experts (past or present) who would be great at answering this?
        Please don't answer the question or comment on it yet."""

    with assistant():
        lm += gen(name='experts', max_tokens=300)
    
    with user():
        lm += f"""\
        Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
        In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
        If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
        Please start your answer with ANSWER:"""
    
    with assistant():
        lm += gen(name='answer', max_tokens=500)

    return lm
                   
mistral_chat + experts(query='What is the meaning of life?')

### Deep Dive on 'stop'

In [64]:
# stop is stopping the generation before the stop value
mistral + "Once upon a time " + gen(name='text', stop=["by"])

In [67]:
# save_stop_text is stopping generation and saving the stop value inside variable named variable_name + "_stop_text"
llm = mistral + "Once upon a time " + gen(name='text', stop=["by"], save_stop_text=True)

In [68]:
llm["text_stop_text"]

'by'

In [69]:
# 2 example of save stop text
llm = mistral + "Once upon a time " + gen(name='example', stop=["by"], save_stop_text=True)

In [70]:
llm["example_stop_text"]

'by'

### JSON formatting

source: https://github.com/guidance-ai/guidance/blob/d36601b62096311988fbba1ba15ae4126fb695df/notebooks/guaranteeing_valid_syntax.ipynb

In [115]:
from guidance import gen, select

# we can pre-define valid option sets
sample_weapons = ["sword", "axe", "mace", "spear", "bow", "crossbow", "wand"]
sample_armor = ["leather", "chainmail", "plate"]

# define a re-usable "guidance function" that we can use below
@guidance
def quoted_list(llm, name, n):
    for i in range(n):
        if i > 0:
            llm += ", "
        llm += '"' + gen(name, list_append=True, stop='"') + '"'
    return llm

@guidance
def generate_character(
    llm,
    character_one_liner,
    weapons: list[str] = sample_weapons,
    armour: list[str] = sample_armor,
    n_items: int = 3
):
    llm += f'''\
    {{
        "description" : "{character_one_liner}",
        "name" : "{gen("character_name", stop='"')}",
        "age" : {gen("age", regex="[2-9][0-9]$")},
        "armour" : "{select(armour, name="armor")}",
        "weapon" : "{select(weapons, name="weapon")}",
        "class" : "{gen("character_class", stop='"')}",
        "mantra" : "{gen("mantra", stop='"')}",
        "strength" : {gen("strength", regex="[1-9][0-9]$")},
        "attack speed" : {gen("speed", regex="[1-9][0-9]$")},
        "accuracy" : {gen("accuracy", regex="[1-9][0-9]$")},
        "damage" : {gen("damage", regex="[1-9][0-9]$")},
        "intelligent" : {gen("intelligent", regex="[1-9][0-9]$")},
        "quest_items" : [{quoted_list("quest_items", n_items)}]
    }}'''
    return llm


result = mistral + generate_character("Strong and tall knight")

In [116]:
# the result are stored in json
import json

gen_json = json.loads(result.__str__())

print(f"Loaded json:\n{json.dumps(gen_json, indent=4)}")

Loaded json:
{
    "description": "Strong and tall knight",
    "name": "Sir Galahad",
    "age": 35,
    "armour": "plate",
    "weapon": "sword",
    "class": "paladin",
    "mantra": "For the greater good",
    "strength": 10,
    "attack speed": 20,
    "accuracy": 80,
    "damage": 15,
    "intelligent": 50,
    "quest_items": [
        "holy_water",
        "cross",
        "amulet"
    ]
}


In [111]:
# we can access it just like dictionary
result["weapon"]

'sword'

JSON formating using chat model

In [12]:
from guidance import models, gen, select, system, user, assistant
import guidance 

mistral_chat = models.LlamaCppChat(
    MODEL_PATH,
    n_ctx=4096,
    n_threads=10
    ) 

# valid order type
order_types = ["BU", "SE", "UNKNOWN"]
integer_regex = r"\d+$" 
stock_code_regex = r"[A-Z][A-Z][A-Z][A-Z]" # not using quantifier of {4} because quantifier is not yet supported

@guidance
def generate_character(
    llm,
    name="order_json",
    order_type: list[str] = order_types,
):
    llm += f'''\
    {{
        "stock_code" : "{gen("stock_code", regex=stock_code_regex)}",
        "order_type" : "{select(order_type, name="order_type")}",
        "quantity" : "{gen("quantity", regex=integer_regex)}",
        "price" : "{gen("price", regex=integer_regex)}"
    }}'''
    return llm

with system():
    llm = mistral_chat + """"
    Extract the following information that is asking for stock news into a JSON string. Your reply should only be a JSON string without any additional comment.
    Ensure that the JSON has the following properties and adhere to the requirements:
            1. Ticker Code (stock_code):
                a. Format: Four capital letters of the company mentioned in the sentence
                b. Default: UNKNOWN
            2. Type of Order (order_type):
                a. Choose one of the following options:
                    BU: Buy a stock
                    SE: Sell a stock
                b. Default: UNKNOWN
            3. Quantity (quantity):
                a. Format: An integer representing the quantity of the order
                b. Default: 0
            4. Price (price):
                a. Format: An integer representing the price of the order
                b. Default: 0
            If no information is available, use the default value.
    """

with user():
    llm += "I want to facebook stock 250 shares"

with assistant():
    json_result_string = llm + generate_character()

In [4]:
print(llm)

[INST] <<SYS>>
"
    Extract the following information that is asking for stock news into a JSON string. Your reply should only be a JSON string without any additional comment.
    Ensure that the JSON has the following properties and adhere to the requirements:
            1. Ticker Code (stock_code):
                a. Format: Four capital letters of the company mentioned in the sentence
                b. Default: UNKNOWN
            2. Type of Order (order_type):
                a. Choose one of the following options:
                    BU: Buy a stock
                    SE: Sell a stock
                b. Default: UNKNOWN
            3. Quantity (quantity):
                a. Format: An integer representing the quantity of the order
                b. Default: 0
            4. Price (price):
                a. Format: An integer representing the price of the order
                b. Default: 0
            If no information is available, use the default value.
    
<</SYS>>

I wa

In [5]:
print(json_result_string)

[INST] <<SYS>>
"
    Extract the following information that is asking for stock news into a JSON string. Your reply should only be a JSON string without any additional comment.
    Ensure that the JSON has the following properties and adhere to the requirements:
            1. Ticker Code (stock_code):
                a. Format: Four capital letters of the company mentioned in the sentence
                b. Default: UNKNOWN
            2. Type of Order (order_type):
                a. Choose one of the following options:
                    BU: Buy a stock
                    SE: Sell a stock
                b. Default: UNKNOWN
            3. Quantity (quantity):
                a. Format: An integer representing the quantity of the order
                b. Default: 0
            4. Price (price):
                a. Format: An integer representing the price of the order
                b. Default: 0
            If no information is available, use the default value.
    
<</SYS>>

I wa

In [53]:
import json

gen_json = json.loads(json_result_string.__str__())

print(f"Loaded json:\n{json.dumps(gen_json, indent=4)}")

JSONDecodeError: Expecting value: line 1 column 2 (char 1)

In [43]:
print(llm['stock_code'])

AAPL


In [44]:
print(llm['quantity'])

250


In [10]:
#ini_list = []
if ini_list:
    print("masuk without not")
if not ini_list:
    print("masuk not")

masuk not


In [29]:
llm + "A word very similiar to 'sky' is " + select(["cloud", "skill"])

### Standalone Question

In [41]:
# determine user query
@guidance
def determine_question(llm, query):
    llm = mistral + f'''\
    Q: {query}
    Determine if the user query is statement, single_question, or context_question.  
    Single question is question that can stand on its own. For example, who is the king of UK? What is the number provinces in germany? 
    context question, on other hand is a question that need additional context, usually but not always containing 'it' or 'they/them/he/she'. 
    For example, who is the king of the country? (because we dont know what country mean in this) 
    or what is the number of provinces in there? (we dont what is 'there' mean). Statement, is obviously anything that are not a question.
    Category: {select(["STATEMENT", "SINGLE_QUESTION", "CONTEXT_QUESTION"], name="question_category")}\n''' 
    if llm["question_category"] == "CONTEXT_QUESTION":
        llm += "need additional context!"
    else:
        llm += query
    return llm

In [46]:
query = "Oke thank you"
mistral + determine_question(query=query)

In [47]:
query = "What is the largest country in the region?"
mistral + determine_question(query=query)

In [49]:
query = "Who is the king of the country?"
mistral + determine_question(query=query)